In [ ]:
# Imports
import sys

sys.path.insert(0, "../..")

from qiskit import QuantumCircuit, qasm2
from qiskit.quantum_info import Clifford, StabilizerState,state_fidelity
from qiskit.visualization import array_to_latex
from qiskit_aer import AerSimulator
import random
from visualisation import LiveVisualizer,GraphWatcher,MatrixWatcher
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Clifford
from qiskit_aer import AerSimulator

from classical_shadow_n_clifford import ClassicalShadow_N_CLIFFORD
from shadow_protocol import ShadowProtocol

In [ ]:
# Setup Experiment
class Id_Protocol(ShadowProtocol):

    def get_num_qubits(self) -> int:
        return 2
    
    def get_state_circuit(self) -> QuantumCircuit:
        circuit = QuantumCircuit(2)
        circuit.x(0)
        return circuit

    def run_circuit_and_get_measurement(self, circuit) -> list[int]:
        sim = AerSimulator()

        # Run with 997 shots
        job = sim.run(circuit, shots=997)
        result = job.result()

        counts = result.get_counts()
        max_hits= max(counts, key=counts.get)
        bit_list =  [int(bit) for bit in list(max_hits)]
        return bit_list[::-1] 
    
id_protocol = Id_Protocol()
# Create Classical Shadow Instance
shadow = ClassicalShadow_N_CLIFFORD(id_protocol)


In [ ]:
vis = LiveVisualizer() # crates the object but nothing will be displayed for the moment

target_density_matrix = shadow.get_original_density_matrix()

fidelity_history = []

while True:

    shadow.add_snapshot()
    
    shadow_size = shadow.get_shadow_size()

    actual_density_matrix = shadow.get_density_matrix_from_cliffords()

    if shadow_size > 100:

        current_fidelity =shadow.calculate_fidelity()
        fidelity_history.append(shadow_size,current_fidelity)
        
        live_matrix_view = MatrixWatcher(
        title_template=f"Live Reconstruction (Shadow size {shadow_size})",
        matrix = actual_density_matrix
        )

        target_matrix_view = MatrixWatcher(
        title_template="Target State (Correct)",
        matrix = target_density_matrix
        )

        fidelity_graph = GraphWatcher(
        title="Fidelity Convergence", 
        target_value=0.5, 
        history=fidelity_history,
        y_min=0.0, 
        y_max=1.0
        )

        vis.update(
            live_matrix_view,    
            target_matrix_view,
            fidelity_graph
        ) # now the view will be updatet and therfore seen on the screen